In [ ]:
# !pip3 install "presidio_analyzer[transformers]"

In [1]:
# https://microsoft.github.io/presidio/analyzer/nlp_engines/transformers/
import transformers
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngineProvider
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
from faker import Faker
import pandas as pd
import json
import re
import spacy


2024-11-02 18:15:04.172335: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-02 18:15:04.173618: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 18:15:04.201630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 18:15:04.707013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Define the model repo
# download a pre trained model
transformers_model = "Jean-Baptiste/camembert-ner"  #"obi/deid_roberta_i2b2" #<PATH_TO_MODEL>
# transformers_model = "flair/ner-french"  #"obi/deid_roberta_i2b2" #<PATH_TO_MODEL>

snapshot_download(repo_id=transformers_model)
# Download pytorch model
# model = AutoModel.from_pretrained(transformers_model)

# Instantiate to make sure it's downloaded during installation and not runtime
AutoTokenizer.from_pretrained(transformers_model)
AutoModelForTokenClassification.from_pretrained(transformers_model)




Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

/home/alejandra/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CamembertForTokenClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=Tr

In [ ]:
#download a spaCy pipeline/model:
# !python -m spacy download en_core_web_sm
# !python -m spacy download fr_core_news_lg

In [16]:
def remove_extra_spaces(txt):
    # remove extra spaces
    txt = " ".join(txt.split())
    txt = txt.replace(" ,", ",").replace(" .", ".")
    # txt = " ".join(txt.split())
    # remove extra simbols
    txt = re.sub(r"([.,])\1+", r"\1", txt)
    return txt
    
# text_to_anonymize=df_all_docPaddel["contents"][7]
text_to_anonymize="""07.JAN.2012 16:42 0386284951 CLINIQUE CLAUDE BERNARD #5103P.001/003 Cacilo ADEMO Dr Annais BRILLIN-BORIS D OLANPOTOC-POMI Dr Sandra AGALI Pathologie Dr Brunetti PELLET wParis. Selaride medecins pathologistes conventionnes 89 bd de Stalin 927A0 MAT.AKOYF 559455880158652233 DR FETRET Christina 66RUEBUTTANT 75007PARIS Pxaman concexaanti MME gONZALEZ Gabriclle NomJF: Nei:01/11/195469as ossierT160285694 du20/11/2011 Noaéjour: opifutns CLINIQUE CLAUDE BERNARD (PMSI) COPIETELEMATIQUE D'UN COMPTE-RENDU J/TUMORECTOMIE SEIN.GAUCHEAYECEXAMENEXTEMPORANE III/IMMUNOHISTOCHIMIE 1/Un ganglion centimetrique est xepére par la sonde isotopique: les appositions per opératoires etPinclusion en paraffine fnontrent qu'il est indemne ca histologie conventionnelle. I1/La tumorectomie mesurant 5 x 3 x 1,5 cm est communiquée orientéc. Reponse extemporanée : foyer tumoral legerement excentre en dedans. Margee mncroacopiquct superieures A 5 mm. Apras inclusion en paraffine, le foyer mesure tur lame mm et comespond a une prolif&ration dc callules cubo cylindtiques a contouts imprecis a cytoplasme peu abondant.Les aoyaux claits ou hyperchrotnatiques pxésentent des variations de diamarre de 1 a 3, un cornpte mitotiquc d'une raitose pour dix champa HPF. Les cellules soat disposécs en travecs d'épaisseut variable au sein d'un stroxa fibreux. I cxiste une composante minoxitaire de carcinome canalaire in situ d'architecture campacte (gradc uterimédiaite saxs uéarose) representant 30% de la susface tumorale. Abseace d'embole vasculite a distance. CONCLUSION Carcinome canalaire infiltrant de 9 mm de grand axe aans cmbole vasculairo. Composante ininorivaire de carcinome canalaire in sinu (grade intermcdiair darchitecture Fompact) repréeentanr 30%de la surface tumorale. Marge microscopique saine la plus étroite de 5 mm en dedana. 3010 Un ganglion sendinclc indemne en hietologic conventionnelle. $BR II fort (3+3+1) index mitouique faible. No/1. Iaaunohistochimie cn coura. L01/03/2012 0386695550 07.JAN.201216:15 RECUDE: 3356-052"""
text_to_anonymize=remove_extra_spaces(text_to_anonymize)

In [17]:
nlp_spacy = spacy.load('fr_core_news_lg')

In [31]:
tokenizer_spacy = nlp_spacy.tokenizer

In [32]:
tokens_sp=tokenizer_spacy(text_to_anonymize)
print(tokens_sp)

07.JAN.2012 16:42 0386284951 CLINIQUE CLAUDE BERNARD #5103P.001/003 Cacilo ADEMO Dr Annais BRILLIN-BORIS D OLANPOTOC-POMI Dr Sandra AGALI Pathologie Dr Brunetti PELLET wParis. Selaride medecins pathologistes conventionnes 89 bd de Stalin 927A0 MAT.AKOYF 559455880158652233 DR FETRET Christina 66RUEBUTTANT 75007PARIS Pxaman concexaanti MME gONZALEZ Gabriclle NomJF: Nei:01/11/195469as ossierT160285694 du20/11/2011 Noaéjour: opifutns CLINIQUE CLAUDE BERNARD (PMSI) COPIETELEMATIQUE D'UN COMPTE-RENDU J/TUMORECTOMIE SEIN.GAUCHEAYECEXAMENEXTEMPORANE III/IMMUNOHISTOCHIMIE 1/Un ganglion centimetrique est xepére par la sonde isotopique: les appositions per opératoires etPinclusion en paraffine fnontrent qu'il est indemne ca histologie conventionnelle. I1/La tumorectomie mesurant 5 x 3 x 1,5 cm est communiquée orientéc. Reponse extemporanée : foyer tumoral legerement excentre en dedans. Margee mncroacopiquct superieures A 5 mm. Apras inclusion en paraffine, le foyer mesure tur lame mm et comespond

In [33]:
# Create configuration containing engine name and models
conf_file = "transformers.yaml"

# Create NLP engine based on configuration
provider = NlpEngineProvider(conf_file=conf_file)
nlp_engine = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine, 
    supported_languages=["fr"]
)

analyzer_results = analyzer.analyze(text=text_to_anonymize, language='fr')



/home/alejandra/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer,EntityRecognizer,RecognizerRegistry
from presidio_analyzer import Pattern
from typing import Optional
from typing import List
# https://github.com/microsoft/presidio/blob/main/presidio-analyzer/presidio_analyzer/predefined_recognizers/email_recognizer.py
class AddressRecognizer(PatternRecognizer):
    """
    Recognize address nb and route using regex and context.

    :param patterns: List of patterns to be used by this recognizer
    :param context: List of context words to increase confidence in detection
    :param supported_language: Language this recognizer supports
    :param supported_entity: The entity this recognizer can detect
    """
    PATTERNS = [
        Pattern(
            "addresses medium",
            r"\b(\d{1,5})\s?(bis|ter|quater|a|b)?", # /b limite de mot
            0.1,
        ),
    ]

    # https://microsoft.github.io/presidio/tutorial/06_context/
    # So this is working, but would catch any 5 digit string. This is why we set the score to 0.01. Let's use context words to increase score:
    CONTEXT =  ["rue",
                "avenue",
                "boulevard",
                "place",
                "chemin",
                "route",
                "impasse",
                "allée",
                "passage", 
                "quai", 
                "bd" ]

    def __init__(
        self,
        patterns: Optional[List[Pattern]] = None,
        context: Optional[List[str]] = None,
        supported_language: str = "fr",
        supported_entity: str = "ADDRESS_NEW",
    ):
        patterns = patterns if patterns else self.PATTERNS
        context = context if context else self.CONTEXT
        super().__init__(
            supported_entity=supported_entity,
            patterns=patterns,
            context=context,
            supported_language=supported_language,
        )

class DoctorRecognizer(PatternRecognizer):
    """
    Recognize doctors names using regex and context.

    :param patterns: List of patterns to be used by this recognizer
    :param context: List of context words to increase confidence in detection
    :param supported_language: Language this recognizer supports
    :param supported_entity: The entity this recognizer can detect
    """
    PATTERNS = [
        Pattern(
            "doc medium",
            r"[A-Z][a-z]+,?\s+(?:[A-Z][a-z]*\.?\s*)?[A-Z][a-z]+", # /b limite de mot  
            0.1,
        ),
    ]

    CONTEXT =  [
        "Dr.",
        "Dr",
        "Docteur",
        "Docteurs",
        "Profeseur",
        "dr",
        "prof.",
        "prof",
    ]

    def __init__(
        self,
        patterns: Optional[List[Pattern]] = None,
        context: Optional[List[str]] = None,
        supported_language: str = "fr",
        supported_entity: str = "DOCTOR_NAME",
    ):
        patterns = patterns if patterns else self.PATTERNS
        context = context if context else self.CONTEXT
        super().__init__(
            supported_entity=supported_entity,
            patterns=patterns,
            context=context,
            supported_language=supported_language,
        )

In [35]:
# https://microsoft.github.io/presidio/analyzer/adding_recognizers/#simple-example
def add_rulest(analyzer):
    titles_list = [
        "Madame",
        "Dr.",
        "Dr",
        "M.",
        "Mme.",
        "Mme",
        "Docteur",
        "Docteurs",
        "Profeseur",
        "mme.",
        "mme",
        "mlle.",
        "monsieur",
        "madame",
        "mademoiselle",
        "mr",
        "mrs",
        "dr",
        "mlle",
        "prof.",
        "prof",
        "patient",
        "patiente"
    ]
    
    titles_recognizer = PatternRecognizer(supported_entity="TITLE", deny_list=titles_list, supported_language="fr")
    # result = titles_recognizer.analyze(text_to_anonymize, entities=["TITLE"])
    # print(f"Result:\n {result}")
    # tt=EntityRecognizer(supported_entities="TITLE", name="ee", supported_language="fr",deny_list=titles_list)
    analyzer.registry.add_recognizer(DoctorRecognizer())
    # analyzer.registry.add_recognizer(titles_recognizer)
    analyzer.registry.add_recognizer(AddressRecognizer())

    return analyzer
    



In [36]:
# analyzer=add_rulest(analyzer)
analyzer_results = analyzer.analyze(text=text_to_anonymize, language='fr')
for e in analyzer_results: 
    print (e,":", text_to_anonymize[e.start:e.end])


type: PERSON, start: 125, end: 137, score: 0.9985368847846985 : Sandra AGALI
type: PERSON, start: 276, end: 292, score: 0.9982286691665649 : FETRET Christina
type: PERSON, start: 152, end: 167, score: 0.9980024099349976 : Brunetti PELLET
type: PERSON, start: 443, end: 457, score: 0.9973115921020508 : CLAUDE BERNARD
type: PERSON, start: 38, end: 52, score: 0.996290385723114 : CLAUDE BERNARD
type: PERSON, start: 340, end: 358, score: 0.9958291053771973 : gONZALEZ Gabriclle
type: PERSON, start: 84, end: 121, score: 0.9939813017845154 : Annais BRILLIN-BORIS D OLANPOTOC-POMI
type: LOCATION, start: 293, end: 316, score: 0.9001020193099976 : 66RUEBUTTANT 75007PARIS
type: LOCATION, start: 168, end: 174, score: 0.8926517367362976 : wParis
type: LOCATION, start: 225, end: 253, score: 0.8730541467666626 : bd de Stalin 927A0 MAT.AKOYF
type: PERSON, start: 68, end: 80, score: 0.8092713356018066 : Cacilo ADEMO
type: LOCATION, start: 1712, end: 1719, score: 0.6620197892189026 : Fompact
type: LOCATION

In [37]:
# Create faker function (note that it has to receive a value)
dictionary_anonymized_people={}
last_name="Gonzalez"
# TODO: when anonimyse person: keep the same fake name
def fake_person(input_text):
    if input_text in dictionary_anonymized_people:
        return dictionary_anonymized_people[input_text]
    elif input_text.find(last_name)>-1:
        new_name=fake.name_female()
        dictionary_anonymized_people[input_text]=new_name
        return new_name
    else:
        new_name=fake.name()
        dictionary_anonymized_people[input_text]=new_name
        return new_name


def fake_location(input_text):
    """
    Parameters:
        input_text (str): The input text from which to generate the fake location.
    Returns:
        str: The generated fake location name or address.
    """
    fake = Faker("fr_FR")
    address_keywords =  ["rue",
                "avenue",
                "boulevard",
                "place",
                "chemin",
                "route",
                "impasse",
                "allée",
                "passage", 
                "quai", 
                "bd" ]
    has_comma = "," in input_text
    if has_comma:
        # Assume it's an address with comma-separated elements
        return " {} ".format(fake.address())

    input_lower = input_text.lower()

    if any(keyword in input_lower for keyword in address_keywords):
        # Generate a fake full address
        return " {} ".format(fake.address())
    # Generate a fake Swiss city name
    fake_city = fake.city()
    # add_name(input_text, "CITY", fake_name)
    return " {} ".format(fake_city)

In [38]:
# Analyzer output
fake = Faker(locale=['fr_FR'])
anonymizer = AnonymizerEngine()
anonymized_text = anonymizer.anonymize(text=text_to_anonymize,
            analyzer_results=analyzer_results,
            operators={"DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}), 
            # "ADDRESS_NUMBER": OperatorConfig("custom", {"lambda": lambda x: fake.address()}),
            "ADDRESS_NUMBER": OperatorConfig("replace", {"new_value": "<ADDRESS_NUMBER>"}), 
            # "PERSON": OperatorConfig("custom", {"lambda": lambda x: fake.name()}),
            "PERSON": OperatorConfig("custom", {"lambda": fake_person}),
            # "LOCATION": OperatorConfig("replace", {"new_value": "<LOCATION>"}), 
            "URL": OperatorConfig("replace", {"new_value": "<URL>"}), 
            "LOCATION": OperatorConfig("custom", {"lambda": fake_location}), 
            "DATE_TIME": OperatorConfig("custom",{"lambda": lambda x: fake.date()}),
           "EMAIL_ADDRESS": OperatorConfig("custom", {"lambda": lambda x: fake.email()}),
           "PHONE_NUMBER": OperatorConfig("custom", {"lambda": lambda x: fake.phone_number()}),
            "TITLE": OperatorConfig("redact", {})}
)
print(f"text: {anonymized_text}")



text: text: 07.JAN.2012 16:42 03 63 14 25 74 CLINIQUE Madeleine Lopez-Briand #5103P.001/003 Aimée Mallet de Guillot Dr Lucy de Benoit Dr Marcel Barbe Pathologie Dr Robert Merle  Olivier .  Benoit-sur-Lamy  medecins pathologistes conventionnes 89  43, rue de Olivier
37285 Andre-les-Bains  559455880158652233 DR Augustin-Rémy Perret  avenue Garcia
53830 Guibert  Pxaman concexaanti MME René Bègue NomJF: Nei:1986-09-08/195469as ossierT160285694 du20/1985-05-23 Noaéjour: opifutns CLINIQUE Madeleine Lopez-Briand (PMSI) COPIETELEMATIQUE D'UN COMPTE-RENDU J/TUMORECTOMIE <URL>UCHEAYECEXAMENEXTEMPORANE III/IMMUNOHISTOCHIMIE 1/Un ganglion centimetrique est xepére par la sonde isotopique: les appositions per opératoires etPinclusion en paraffine fnontrent qu'il est indemne ca histologie conventionnelle. I1/La tumorectomie mesurant 5 x 3 x 1,5 cm est communiquée orientéc. Reponse extemporanée : foyer tumoral legerement excentre en dedans. Margee mncroacopiquct superieures A 5 mm. Apras inclusion en 

In [26]:
print(anonymized_text.text)

07.JAN.2012 16:42 03 66 87 55 19 CLINIQUE Zacharie Legendre #5103P.001/003 Grégoire Tanguy Dr Michèle-Juliette Leleu Dr Timothée Fernandes du Maillet Pathologie Dr Céline Étienne de la Lemaître  Saint MatthieuBourg .  Petitjeannec  medecins pathologistes conventionnes 89  4, rue de Boyer
97265 Laine-la-Forêt  559455880158652233 DR Benoît du Duhamel  17, rue Margot Boucher
80825 Rocher  Pxaman concexaanti MME Benoît Guillet NomJF: Nei:2001-03-03/195469as ossierT160285694 du20/2008-04-01 Noaéjour: opifutns CLINIQUE Zacharie Legendre (PMSI) COPIETELEMATIQUE D'UN COMPTE-RENDU J/TUMORECTOMIE <URL>UCHEAYECEXAMENEXTEMPORANE III/IMMUNOHISTOCHIMIE 1/Un ganglion centimetrique est xepére par la sonde isotopique: les appositions per opératoires etPinclusion en paraffine fnontrent qu'il est indemne ca histologie conventionnelle. I1/La tumorectomie mesurant 5 x 3 x 1,5 cm est communiquée orientéc. Reponse extemporanée : foyer tumoral legerement excentre en dedans. Margee mncroacopiquct superieures A

In [27]:
# https://github.com/microsoft/presidio/issues/1083
# from presidio_analyzer import AnalyzerEngine, RecognizerRegistry
# from transformers_recognizer import TransformersRecognizer
# import spacy
# from presidio_analyzer.nlp_engine import NlpEngineProvider

# FR_MODEL_CONF = {'PRESIDIO_SUPPORTED_ENTITIES': ['LOCATION', 'PERSON', 'ORGANIZATION', 'DATE_TIME', 'NRP'],
#                'DEFAULT_MODEL_PATH': 'Jean-Baptiste/camembert-ner-with-dates',
#                'DATASET_TO_PRESIDIO_MAPPING': {'DATE': 'DATE_TIME', 'MISC': 'NRP', 'PER': 'PERSON', 'ORG': 'ORGANIZATION', 'LOC': 'LOCATION'},
#                "MODEL_TO_PRESIDIO_MAPPING": {'DATE': 'DATE_TIME', 'MISC': 'NRP', 'PER': 'PERSON', 'ORG': 'ORGANIZATION', 'LOC': 'LOCATION'},
#                "CHUNK_OVERLAP_SIZE": 40,
#                "CHUNK_SIZE": 600,
#                "ID_SCORE_MULTIPLIER": 0.4,
#                "ID_ENTITY_NAME": "ID"}

# registry = RecognizerRegistry()
# registry.load_predefined_recognizers()

# if not spacy.util.is_package("fr_core_news_sm"):
#    spacy.cli.download("fr_core_news_sm")


# supported_entities = FR_MODEL_CONF.get(
#        "PRESIDIO_SUPPORTED_ENTITIES")

# model = "Jean-Baptiste/camembert-ner-with-dates"
# transformers_recognizerr = TransformersRecognizer(model_path=model, supported_entities= supported_entities)
# transformers_recognizerr.load_transformer(**FR_MODEL_CONF)

# if not spacy.util.is_package("fr_core_news_sm"):
#    spacy.cli.download("fr_core_news_sm")

# registry.add_recognizer(transformers_recognizerr)
# registry.remove_recognizer("SpacyRecognizer")


# nlp_configuration = {
#    "nlp_engine_name": "spacy",
#    "models": [{"lang_code": "fr", "model_name":"fr_core_news_sm"}],
# }

# nlp_engine = NlpEngineProvider(nlp_configuration=nlp_configuration).create_engine()

# analyzer = AnalyzerEngine(registry=registry, nlp_engine=nlp_engine)

# results = analyzer.analyze(
# text="Je m'appelle jean-baptiste et j'habite à montréal depuis fevr 2012",
# language="fr",
# entities = ['LOCATION', 'PERSON', 'ORGANIZATION', 'DATE_TIME', 'NRP'],
# return_decision_process=True,
# )
# for result in results:
#    print(result)
#    print(result.analysis_explanation)